In [1]:
global Path
if sc.master[0:5]=="local":
    Path="file:/home/hduser/pythonwork/PythonProject"
else:
    Path="hdfs://master:9000/user/hduser/"


In [2]:
rawUserData=sc.textFile(Path+"data/u.data")
rawUserData.count()


100000

In [3]:
rawUserData.first()


u'196\t242\t3\t881250949'

In [4]:
from pyspark.mllib.recommendation import Rating


In [5]:
rawRatings = rawUserData.map(lambda line: line.split("\t")[:3])
rawRatings.take(5)


[[u'196', u'242', u'3'],
 [u'186', u'302', u'3'],
 [u'22', u'377', u'1'],
 [u'244', u'51', u'2'],
 [u'166', u'346', u'1']]

In [6]:
ratingsRDD=rawRatings.map(lambda x: (x[0], x[1], x[2]))
ratingsRDD.take(5)


[(u'196', u'242', u'3'),
 (u'186', u'302', u'3'),
 (u'22', u'377', u'1'),
 (u'244', u'51', u'2'),
 (u'166', u'346', u'1')]

In [7]:
numRatings = ratingsRDD.count()
numRatings


100000

In [8]:
numUsers = ratingsRDD.map(lambda x: x[0]).distinct().count()
numUsers


943

In [9]:
numMovies = ratingsRDD.map(lambda x: x[1]).distinct().count()
numMovies


1682

In [10]:
from pyspark.mllib.recommendation import ALS


In [11]:
model=ALS.train(ratingsRDD, 10, 10, 0.01)


In [12]:
print(model)


In [13]:
model.recommendProducts(100, 5)


[Rating(user=100, product=1268, rating=5.512395179257409),
 Rating(user=100, product=1449, rating=5.202799371618108),
 Rating(user=100, product=1120, rating=5.1502823060211735),
 Rating(user=100, product=1169, rating=5.040170504690009),
 Rating(user=100, product=1446, rating=5.00262771367998)]

In [14]:
model.predict(100,1141)


2.5108042105022377

In [15]:
model.recommendUsers(product=200, num=5)


[Rating(user=475, product=200, rating=6.57450570542799),
 Rating(user=762, product=200, rating=5.790050493995605),
 Rating(user=849, product=200, rating=5.787045358614803),
 Rating(user=512, product=200, rating=5.754979611444351),
 Rating(user=938, product=200, rating=5.616582327569422)]

In [16]:
itemRDD = sc.textFile(Path + "data/u.item")
itemRDD.count()


1682

In [17]:
movieTitle=itemRDD.map(lambda line : line.split("|")).map(lambda a : (float(a[0]),a[1])).collectAsMap()
len(movieTitle)


1682

In [18]:
movieTitle.items()[:5]


[(1.0, u'Toy Story (1995)'),
 (2.0, u'GoldenEye (1995)'),
 (3.0, u'Four Rooms (1995)'),
 (4.0, u'Get Shorty (1995)'),
 (5.0, u'Copycat (1995)')]

In [19]:
movieTitle[5]


u'Copycat (1995)'

In [20]:
recommendP=model.recommendProducts(100,5)
for p in recommendP:
    print "for user: " + str(p[0]) + " recommend: " + str(movieTitle[p[1]]) + " score: " + str(p[2])


for user: 100 recommend: Bitter Moon (1992) score: 5.51239517926
for user: 100 recommend: Pather Panchali (1955) score: 5.20279937162
for user: 100 recommend: I'm Not Rappaport (1996) score: 5.15028230602
for user: 100 recommend: Fresh (1994) score: 5.04017050469
for user: 100 recommend: Bye Bye, Love (1995) score: 5.00262771368
